# K-Nearest Neighbor

### Part 1: Implementation

For this assignment, I'm going to let you break down the implementation as you see fit. You're going to implement KNN. In brief, this means:

- calculating euclidean distance between a test instance, and a training instance
- iterating through all the training instances, and storing distances in a list
- returning the k nearest neighbors
- making a prediction by choosing the class that appears the most in the k nearest neighbors

To calculate euclidean distance between an instance x1 and an instance x2, we need to iterate through the features (excluding the class) of the two instances (for i features) and for each take the difference of (x1[i]) - (x2[i]), squaring that difference, and summing over all features. At the end, take the square root of the total. In other words:

$$distance=\sqrt{\sum_{i=1}^n (x1_{i} - x2_{i})^2}$$

I didn't really need to include this equation, but now you have an example of embedding a latex equation inside markdown. You're welcome.

I would strongly suggest you follow the implementation outline of previous algorithms in terms of the functions you use, but I'm leaving it up to you.

Below is the same contrived dataset you've used before. If your code works, you should be able to take an instance of this data, and compare it to all the others (including itself, where the distance SHOULD be 0). You should be able to select the k-nearest neighbors, and make a prediction based on the most frequently occuring class.

Make sure you create a knn function that takes a training set, a test set and a value for k.


In [1]:
import math
import collections
from collections import Counter 

# Contrived data set

dataset = [[3.393533211,2.331273381,0],
    [3.110073483,1.781539638,0],
    [1.343808831,3.368360954,0],
    [3.582294042,4.67917911,0],
    [2.280362439,2.866990263,0],
    [7.423436942,4.696522875,1],
    [5.745051997,3.533989803,1],
    [9.172168622,2.511101045,1],
    [7.792783481,3.424088941,1],
    [7.939820817,0.791637231,1]]

def euclid(train_inst, test_inst):
    inst_len = len(train_inst)-1
    dist = 0
    for i in range(inst_len):
        dist += (train_inst[i] - test_inst[i])**2
    return math.sqrt(dist)
    
def nearest_neighbor(train, test_inst, k):
    dist_list = []
    for i in range(len(train)):       
        dist_list.append([euclid(train[i], test_inst), train[i][-1]])
    dist_list.sort(key = lambda dist: dist[0])
    knn = []
    for i in range(k):
        knn.append(dist_list[i][-1])
    return knn

def knn_class(train, test, k):
    predict = []
    for instance in test:
        knn = nearest_neighbor(train, instance, k)
        predict.append(Counter(knn).most_common(1)[0][0])
    return predict 

print(knn_class(dataset, dataset, 3))

[0, 0, 0, 0, 0, 1, 1, 1, 1, 1]


### Part 2: Working with real data

Apply the KNN algorithm above to the abalone data set. You can find more about it here: http://archive.ics.uci.edu/ml/datasets/Abalone

You will need to make the class value into an integer class. Run a 5-fold cross-validation, with k set as 5. Also run a classification baseline. Report on classification accuracy, and write up some results.

In [2]:
import csv 
import copy
import random

filename = 'abalone.csv'

def cross_validation_data(dataset, folds):
    new_list = []
    copy_list = copy.deepcopy(dataset)
    fold_len = len(dataset)/folds
    for i in range(folds):
        current_fold = []
        while len(current_fold) < fold_len and len(copy_list)!=0:
            random_inst = random.choice(copy_list) 
            current_fold.append(random_inst)
            copy_list.remove(random_inst)
        new_list.append(current_fold)
    return new_list

def evaluate_algorithm(dataset, algorithm, folds, metric, *args):
    new_data = cross_validation_data(dataset, folds)  
    scores = []
    for fold in new_data:
        train = copy.deepcopy(new_data)
        train.remove(fold)
        train = [element for sublist in train for element in sublist]
        test = [instance[:-1] + [None] for instance in fold]
        
        predicted = algorithm(train,test, *args)
        actual = [instance[-1] for instance in fold]
        result = metric(actual,predicted)
        scores.append(result)

    return scores

#=======================================================
def load_data(filename):
    csv_reader = csv.reader(open(filename, newline=''), delimiter=',')
    new_list = []
    for row in csv_reader:
        new_list.append(row)
    return new_list

data_abalone = load_data(filename)

# Convert the features from strings to floats 
def column2Float(dataset, column):
    for row in dataset:
        row[column] = float(row[column])
        
for row in data_abalone:
    if row[0] == 'M':
        row[0] = 0
    elif row[0] == 'F':
        row[0] = 1
    else:
        row[0] = 2
        
for i in range(1,len(data_abalone[0])):  
    column2Float(data_abalone, i)  

def mean(listOfValues):
    return sum(listOfValues)/len(listOfValues)

#==============================================
def accuracy(actual, predicted):
    length = len(actual)
    counter = 0
    for i in range(length):
        if actual[i] == predicted[i]:
            counter += 1
    return (counter/length)*100

def zeroRC(train, test):
    valueY = [instance[-1] for instance in train]
    most_occur = Counter(valueY).most_common(1)[0][0] 
    return [most_occur for i in range(len(test))]

#==============================================
folds = 5
k = 10

KNN_scores = evaluate_algorithm(data_abalone, knn_class, folds, accuracy, k)
zeroRC_scores = evaluate_algorithm(data_abalone, zeroRC, folds, accuracy)

print("\nNumber of instances:", len(data_abalone))
print("Number of features :", len(data_abalone[0])-1, "\n")

print("KNN highest accuracy:", max(KNN_scores))
print("KNN lowest accuracy:", min(KNN_scores))
print("KNN mean accuracy:", mean(KNN_scores), "\n")

print("zeroRC highest accuracy:", max(zeroRC_scores))
print("zeroRC lowest accuracy:", min(zeroRC_scores))
print("zeroRC mean accuracy:", mean(zeroRC_scores))


Number of instances: 4177
Number of features : 8 

KNN highest accuracy: 27.392344497607656
KNN lowest accuracy: 23.086124401913878
KNN mean accuracy: 25.185212841117306 

zeroRC highest accuracy: 17.3444976076555
zeroRC lowest accuracy: 14.765906362545017
zeroRC mean accuracy: 16.493851128968334


#### Write up results here

- KNN performs better than the baseline, however, the accuracy rate is not very high. This may be due to the fact that the class (Age) varies widely (1-29 yrs old) and number of neighbors (k=5) is not enough to capture the right class. 

### Part 3: KNN regression

We can also run KNN as a regression algorithm. In this case, instead of predicting the most common class in the k nearest neighbors, we can assign a predicted value that is the mean of the values in the k neighbors. 

Make this change to your algorithm (presumably by simply implementing a new predict function below, because you divided your code up sensibly in Part 1), and run the abalone data as a regression problem (convert the class data to a float, before running the algorithm). Use the same number of folds and the same k value as before. Also run a regression baseline and report RMSE values for both. Give me some explanation of the results, both standalone and in comparison to the classification results above.


In [3]:
def knn_reg(train, test, k):
    predict = []
    for instance in test:
        knn = nearest_neighbor(train, instance, k)
        predict.append(mean(knn))
    return predict 

def zeroRR(train, test):
    targetList = [instance[len(instance)-1] for instance in train]
    targetMean = mean(targetList)
    prediction = [targetMean for i in range(len(train))]
    return prediction
        
def rmse_eval(actual, predicted):
    error = 0.0
    length = len(actual)
    for i in range(length):
        error += (predicted[i] - actual[i])**2
    error = (error/length)**0.5
    return error

#==============================================
data_abalone = load_data(filename)

for row in data_abalone:
    if row[0] == 'M':
        row[0] = 0
    elif row[0] == 'F':
        row[0] = 1
    else:
        row[0] = 2
        
for i in range(len(data_abalone[0])):  
    column2Float(data_abalone, i)

folds = 5
k = 5

KNN_scores = evaluate_algorithm(data_abalone, knn_reg, folds, rmse_eval, k)
zeroRR_scores = evaluate_algorithm(data_abalone, zeroRR, folds, rmse_eval)

print("\nNumber of instances:", len(data_abalone))
print("Number of features :", len(data_abalone[0])-1, "\n")

print("KNN highest error:", max(KNN_scores))
print("KNN lowest error:", min(KNN_scores))
print("KNN mean error:", mean(KNN_scores), "\n")

print("zeroRR highest error:", max(zeroRR_scores))
print("zeroRR lowest error:", min(zeroRR_scores))
print("zeroRR mean error:", mean(zeroRR_scores))


def arrangingRules(rules):
    # Write your code here

if __name__ == '__main__':
    fptr = open(os.environ['OUTPUT_PATH'], 'w')

    rules_count = int(input().strip())

    rules = []

    for _ in range(rules_count):
        rules_item = input()
        rules.append(rules_item)

    result = arrangingRules(rules)

    fptr.write('\n'.join(result))
    fptr.write('\n')

    fptr.close()



Number of instances: 4177
Number of features : 8 

KNN highest error: 2.352113983502406
KNN lowest error: 2.1812911801235964
KNN mean error: 2.2642945227933895 

zeroRR highest error: 3.3405832728341984
zeroRR lowest error: 3.089706040475734
zeroRR mean error: 3.2234056127149393


#### Write up results here

KNN performs nicely with error rate around 2. However, this is not much higher than that of zeroRR which may be due to the fact that most inputs have age clusters around the mean (age 6-11 make up 3028 instances, 3/4 of the dataset).

This method of assigning mean values works better than the previous one which attempts to make exact prediction. Again, this is because age varies in a wide range, making it hard to predict the exact age but easier to predict a value that is close enough to the true one.